# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Sep 6 2022 10:12AM,245713,15,8475593,"Brookfield Pharmaceuticals, LLC",Released
1,Sep 6 2022 10:12AM,245713,15,8475594,"Brookfield Pharmaceuticals, LLC",Released
2,Sep 6 2022 10:12AM,245713,15,8475595,"Brookfield Pharmaceuticals, LLC",Released
3,Sep 6 2022 10:12AM,245713,15,8475596,"Brookfield Pharmaceuticals, LLC",Released
4,Sep 6 2022 10:12AM,245713,15,8475597,"Brookfield Pharmaceuticals, LLC",Released
5,Sep 6 2022 10:12AM,245713,15,8475598,"Brookfield Pharmaceuticals, LLC",Released
6,Sep 6 2022 10:12AM,245713,15,8475599,"Brookfield Pharmaceuticals, LLC",Released
7,Sep 6 2022 10:12AM,245713,15,8475600,"Brookfield Pharmaceuticals, LLC",Released
8,Sep 6 2022 10:12AM,245713,15,8475601,"Brookfield Pharmaceuticals, LLC",Released
9,Sep 6 2022 10:12AM,245713,15,8475602,"Brookfield Pharmaceuticals, LLC",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
18,245709,Released,60
19,245710,Released,55
20,245711,Released,67
21,245712,Released,1
22,245713,Released,42


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
245709,NaN,NaN,60.0
245710,NaN,NaN,55.0
245711,NaN,NaN,67.0
245712,NaN,NaN,1.0
245713,NaN,NaN,42.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
245682,0.0,0.0,4.0
245684,2.0,11.0,1.0
245689,0.0,0.0,7.0
245693,0.0,0.0,64.0
245694,0.0,32.0,16.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
245682,0,0,4
245684,2,11,1
245689,0,0,7
245693,0,0,64
245694,0,32,16


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,245682,0,0,4
1,245684,2,11,1
2,245689,0,0,7
3,245693,0,0,64
4,245694,0,32,16


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,245682,,,4
1,245684,2,11,1
2,245689,,,7
3,245693,,,64
4,245694,,32,16


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Sep 6 2022 10:12AM,245713,15,"Brookfield Pharmaceuticals, LLC"
42,Sep 6 2022 10:09AM,245711,10,ISDIN Corporation
109,Sep 6 2022 10:07AM,245710,10,ISDIN Corporation
164,Sep 6 2022 10:06AM,245709,10,ISDIN Corporation
224,Sep 6 2022 9:59AM,245712,16,"Emersa Waterbox, LLC"
225,Sep 6 2022 9:56AM,245703,10,ISDIN Corporation
286,Sep 6 2022 9:56AM,245707,20,"Exact-Rx, Inc."
300,Sep 6 2022 9:54AM,245706,20,Alumier Labs Inc.
304,Sep 6 2022 9:47AM,245705,10,ISDIN Corporation
306,Sep 6 2022 9:42AM,245704,12,Hush Hush


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Sep 6 2022 10:12AM,245713,15,"Brookfield Pharmaceuticals, LLC",,,42
1,Sep 6 2022 10:09AM,245711,10,ISDIN Corporation,,,67
2,Sep 6 2022 10:07AM,245710,10,ISDIN Corporation,,,55
3,Sep 6 2022 10:06AM,245709,10,ISDIN Corporation,,,60
4,Sep 6 2022 9:59AM,245712,16,"Emersa Waterbox, LLC",,,1
5,Sep 6 2022 9:56AM,245703,10,ISDIN Corporation,,,61
6,Sep 6 2022 9:56AM,245707,20,"Exact-Rx, Inc.",,,14
7,Sep 6 2022 9:54AM,245706,20,Alumier Labs Inc.,,,4
8,Sep 6 2022 9:47AM,245705,10,ISDIN Corporation,,,2
9,Sep 6 2022 9:42AM,245704,12,Hush Hush,,1,


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 6 2022 10:12AM,245713,15,"Brookfield Pharmaceuticals, LLC",42,,
1,Sep 6 2022 10:09AM,245711,10,ISDIN Corporation,67,,
2,Sep 6 2022 10:07AM,245710,10,ISDIN Corporation,55,,
3,Sep 6 2022 10:06AM,245709,10,ISDIN Corporation,60,,
4,Sep 6 2022 9:59AM,245712,16,"Emersa Waterbox, LLC",1,,
5,Sep 6 2022 9:56AM,245703,10,ISDIN Corporation,61,,
6,Sep 6 2022 9:56AM,245707,20,"Exact-Rx, Inc.",14,,
7,Sep 6 2022 9:54AM,245706,20,Alumier Labs Inc.,4,,
8,Sep 6 2022 9:47AM,245705,10,ISDIN Corporation,2,,
9,Sep 6 2022 9:42AM,245704,12,Hush Hush,,1,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 6 2022 10:12AM,245713,15,"Brookfield Pharmaceuticals, LLC",42,,
1,Sep 6 2022 10:09AM,245711,10,ISDIN Corporation,67,,
2,Sep 6 2022 10:07AM,245710,10,ISDIN Corporation,55,,
3,Sep 6 2022 10:06AM,245709,10,ISDIN Corporation,60,,
4,Sep 6 2022 9:59AM,245712,16,"Emersa Waterbox, LLC",1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 6 2022 10:12AM,245713,15,"Brookfield Pharmaceuticals, LLC",42.0,NaN,NaN
1,Sep 6 2022 10:09AM,245711,10,ISDIN Corporation,67.0,NaN,NaN
2,Sep 6 2022 10:07AM,245710,10,ISDIN Corporation,55.0,NaN,NaN
3,Sep 6 2022 10:06AM,245709,10,ISDIN Corporation,60.0,NaN,NaN
4,Sep 6 2022 9:59AM,245712,16,"Emersa Waterbox, LLC",1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 6 2022 10:12AM,245713,15,"Brookfield Pharmaceuticals, LLC",42.0,0.0,0.0
1,Sep 6 2022 10:09AM,245711,10,ISDIN Corporation,67.0,0.0,0.0
2,Sep 6 2022 10:07AM,245710,10,ISDIN Corporation,55.0,0.0,0.0
3,Sep 6 2022 10:06AM,245709,10,ISDIN Corporation,60.0,0.0,0.0
4,Sep 6 2022 9:59AM,245712,16,"Emersa Waterbox, LLC",1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,2948399,437.0,43.0,2.0
12,491406,56.0,1.0,0.0
15,245713,42.0,0.0,0.0
16,245712,1.0,0.0,0.0
20,982784,29.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,2948399,437.0,43.0,2.0
1,12,491406,56.0,1.0,0.0
2,15,245713,42.0,0.0,0.0
3,16,245712,1.0,0.0,0.0
4,20,982784,29.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,437.0,43.0,2.0
1,12,56.0,1.0,0.0
2,15,42.0,0.0,0.0
3,16,1.0,0.0,0.0
4,20,29.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,437.0
1,12,Released,56.0
2,15,Released,42.0
3,16,Released,1.0
4,20,Released,29.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,20
Status,,,,,
Completed,2.0,0.0,0.0,0.0,0.0
Executing,43.0,1.0,0.0,0.0,0.0
Released,437.0,56.0,42.0,1.0,29.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,20
0,Completed,2.0,0.0,0.0,0.0,0.0
1,Executing,43.0,1.0,0.0,0.0,0.0
2,Released,437.0,56.0,42.0,1.0,29.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,20
0,Completed,2.0,0.0,0.0,0.0,0.0
1,Executing,43.0,1.0,0.0,0.0,0.0
2,Released,437.0,56.0,42.0,1.0,29.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()